In [1]:
import os


In [2]:
%pwd

'F:\\FAHIM\\Data Science\\y_ml_project\\p2\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'F:\\FAHIM\\Data Science\\y_ml_project\\p2'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelBaseConfig:
    root_dir: Path
    base_model_path: Path
    update_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [29]:
from src.chicken_deases_classification.constant import *
from src.chicken_deases_classification.utils.common import read_yaml,create_directories

In [30]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath =PARAMS_FILE_PATH):
        self.config =read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model_config(self) -> PrepareModelBaseConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareModelBaseConfig(
            root_dir = Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            update_base_model_path=Path(config.update_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES)
        return prepare_base_model_config
        

In [31]:
# Components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [37]:
class PrepareBaseModel:
    def __init__(self,config :PrepareModelBaseConfig):
        self.config = config
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape =self.config.params_image_size,
        weights=self.config.params_weights,
        include_top=self.config.params_include_top)
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_all > 0):
            for layer in model.layers[:freeze_all]:
                model.trainable = False
        flatten_in =tf.keras.layers.Flatten()(model.output)
        predictions =tf.keras.layers.Dense(units=classes,activation='softmax')(flatten_in)
        full_model = tf.keras.Model(inputs=model.input,outputs=predictions)
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
        
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.update_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path : Path, model : tf.keras.Model):
        model.save(path)
        

In [38]:
# Pipeline
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-12 21:55:30,715: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-12 21:55:30,715: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-12 21:55:30,715: INFO: common: created directory at: artifacts]
[2024-07-12 21:55:30,715: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-07-12 21:55:31,941: WARNING: module_wrapper: From C:\Users\Maaz Computer\anaconda3\ana\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-07-12 21:55:33,510: WARNING: module_wrapper: From C:\Users\Maaz Computer\anaconda3\ana\Lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
[2024-07-12 21:55:35,242: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will 

C:\Users\Maaz Computer\anaconda3\ana\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     